# Depthwise-seperable 2D Convolution on GPU

## 1. Set-up 

In [1]:
# Mount google drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Make sure your token is stored in a txt file at the location below.
# This way there is no risk that you will push it to your repo
# Never share your token with anyone, it is basically your github password!
with open('/content/drive/MyDrive/ece5545/token.txt') as f:
    token = f.readline().strip()
# Use another file to store your github username    
with open('/content/drive/MyDrive/ece5545/git_username.txt') as f:
    handle = f.readline().strip()

In [26]:
# Clone your github repo
YOUR_TOKEN = token
YOUR_HANDLE = handle
BRANCH = "main"

%mkdir /content/drive/MyDrive/ece5545
%cd /content/drive/MyDrive/ece5545
!git clone https://{YOUR_TOKEN}@github.com/ML-HW-SYS/a3-{YOUR_HANDLE}.git
%cd /content/drive/MyDrive/ece5545/a3-{YOUR_HANDLE}
!git checkout {BRANCH}
!git pull
%cd /content/drive/MyDrive/ece5545

PROJECT_ROOT = f"/content/drive/MyDrive/ece5545/a3-{YOUR_HANDLE}"

mkdir: cannot create directory ‘/content/drive/MyDrive/ece5545’: File exists
/content/drive/MyDrive/ece5545
fatal: destination path 'a3-NamanMakkar' already exists and is not an empty directory.
/content/drive/MyDrive/ece5545/a3-NamanMakkar
Already on 'main'
Your branch is up to date with 'origin/main'.
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (4/4), 871 bytes | 1024 bytes/s, done.
From https://github.com/ML-HW-SYS/a3-NamanMakkar
   bc7521a..6e3ad80  main       -> origin/main
Updating bc7521a..6e3ad80
Fast-forward
 src/ops.py | 8 ++++++++
 1 file changed, 8 insertions(+)
/content/drive/MyDrive/ece5545


In [7]:
# This extension reloads all imports before running each cell
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
!ls {PROJECT_ROOT}

1-conv1d_cpu.ipynb   4-gemm_gpu.ipynb	    README.md
2-conv1d_gpu.ipynb   5-conv2d_dw_gpu.ipynb  src
3-conv1d_fpga.ipynb  leaderboard_id.txt     tests


## 2 Install TVM

In [6]:
!pip install tlcpack-nightly-cu102 -f https://tlcpack.ai/wheels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://tlcpack.ai/wheels
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 MB 3.7 MB/s eta 0:00:00


## 3. Implement `make_dwsp_conv2d_gpu_scheduler` function in `src.ops`

In that function, you are required to implemented 2D convolution and use TVM to optimize it.
Please use zero padding and unit stride. 
You can assume kernel size to be an odd number.
The padding will equals to kernel size minus ones.
In this case, the output image will preserve the input image dimension.

The `make_dwsp_conv2d_gpu_scheduler` takes following arguments:
1. Batch size $B$;
2. Input channel size $C$;
3. Input image height $H$;
4. Input image width $W$;
5. Output number of channels $O$;
6. Kernel size $K$

You should return both the TVM scheduler and the TVM opterator for 
1. Input tensor $x$ with size (B, C, H, W)
2. Input kernel weight $y$ with size (O, 1, K, K)
3. Output $out$ with size (B, O, H, W)

The scheduler should be able to used to build a function with signature $func(x, y, out)$. 
Please see the following cells the usage.

In [30]:
import tvm
import torch.nn.functional as F
import numpy as np
import sys
import os
import tvm
from tvm import te
# Adding assignment 3 to the system path
# Make sure this matches your git directory
sys.path.insert(0, PROJECT_ROOT)
from src.ops import make_dwsp_conv2d_gpu_scheduler

B = 3
C = 4
H = 16
W = 32
#O = C
K = 7
dtype = 'float32'
a_np = np.random.rand(B, C, H, W).astype(dtype)
w_np = np.random.rand(C, 1, K, K).astype(dtype)

s, inp, ker, out = make_dwsp_conv2d_gpu_scheduler(B, C, H, W, K) 
func = tvm.build(s, [inp, ker, out], "cuda")

dev = tvm.cuda(0)
a = tvm.nd.array(a_np, dev)
w = tvm.nd.array(w_np, dev)
b = tvm.nd.array(np.zeros((B, C, H, W), dtype), dev)
func(a, w, b)
evaluator = func.time_evaluator(func.entry_name, dev, number=1, repeat =1)

print("Output:", b)
print(f"2DConv TVM: %f ms" % (evaluator(a, w, b).mean * 1e3))

Output: [[[[ 5.992046   6.6133723  6.767883  ...  7.879144   6.1353607
     5.5820565]
   [ 7.0219145  7.1709795  7.787781  ...  8.748227   7.108428
     6.759547 ]
   [ 7.164562   7.8462653  8.829841  ...  9.635308   8.22555
     6.439999 ]
   ...
   [ 5.41565    8.350394   8.997432  ...  9.640601   7.5656223
     5.4365196]
   [ 5.357828   7.3932004  7.518719  ...  7.3347936  6.2817135
     5.290862 ]
   [ 3.2647572  4.66572    5.8500423 ...  5.6427293  4.828835
     3.5719402]]

  [[ 4.8873014  4.392408   5.7115483 ...  3.3302038  2.800436
     3.0503962]
   [ 5.376614   5.626443   6.020023  ...  4.2395353  3.5958667
     3.3231676]
   [ 5.586133   6.6196575  7.3366265 ...  4.799673   4.5222287
     5.0465994]
   ...
   [ 5.336596   6.968148   9.311557  ...  7.370347   5.598652
     4.276941 ]
   [ 5.571346   6.8386207  7.4686933 ...  6.6963224  5.078533
     4.226414 ]
   [ 3.931772   4.624997   5.9025626 ...  5.239812   4.5820704
     3.3875384]]

  [[ 4.0319195  5.330878   5.5825

In [31]:
print(tvm.lower(s, [inp, ker, out], simple_mode=True))

# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((3, 4, 16, 32), "float32"), W: T.Buffer((4, 1, 7, 7), "float32"), out: T.Buffer((3, 4, 16, 32), "float32")):
        T.func_attr({"from_legacy_te_schedule": T.bool(True), "global_symbol": "main", "tir.noalias": T.bool(True)})
        padded_inp = T.allocate([10032], "float32", "global")
        padded_inp_1 = T.Buffer((10032,), data=padded_inp)
        for i0, i1 in T.grid(3, 4):
            blockIdx_z = T.launch_thread("blockIdx.z", 6)
            blockIdx_y = T.launch_thread("blockIdx.y", 10)
            threadIdx_z = T.launch_thread("threadIdx.z", 4)
            threadIdx_y = T.launch_thread("threadIdx.y", 4)
            if T.likely(blockIdx_z * 2 + threadIdx_z // 2 < 11):
                if T.likely(blockIdx_y * 2 + threadIdx_y // 2 < 19):
                    A_1 = T.Buffer((6144,), data=A.data)
                    padded_inp_1[i0 * 3344 + i1 * 83

In [32]:
%cd {PROJECT_ROOT}
!python -m pytest tests/test_dwsp_2dconv_gpu.py

/content/drive/MyDrive/ece5545/a3-NamanMakkar
============================= test session starts ==============================
platform linux -- Python 3.9.16, pytest-7.2.2, pluggy-1.0.0
rootdir: /content/drive/MyDrive/ece5545/a3-NamanMakkar
plugins: anyio-3.6.2
collected 1357 items                                                           

tests/test_dwsp_2dconv_gpu.py .......................................... [  3%]
........................................................................ [  8%]
........................................................................ [ 13%]
........................................................................ [ 19%]
........................................................................ [ 24%]
........................................................................ [ 29%]
........................................................................ [ 34%]
........................................................................ [ 40%]
................